# Script d'inference
---
<a target="_blank" href="https://colab.research.google.com/drive/1FxU8SOoghUwyI-Eza_gPWllHSXyVok5k">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

https://huggingface.co/docs/transformers/model_doc/segformer

## Connect do GoogleDrive

---



In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


## Install dependencies

---



In [2]:
%%capture
! pip install split-folders
! pip install evaluate
! pip install rasterio
! pip install onnx
! pip install onnxruntime-gpu
! pip install --quiet bitsandbytes
! pip install --quiet --upgrade transformers # Install latest version of transformers
! pip install --quiet --upgrade accelerate
! pip install onnxconverter-common

In [3]:
import accelerate
import transformers
import torch

print(transformers.__version__, accelerate.__version__)
print(torch.__version__)

from torch.utils.data import Dataset, DataLoader
from transformers import AdamW
from torch import nn
from sklearn.metrics import accuracy_score
from tqdm.notebook import tqdm
import os
from PIL import Image
from transformers import SegformerForSemanticSegmentation, SegformerImageProcessor,SegformerFeatureExtractor
import pandas as pd
import cv2
import numpy as np
import albumentations as aug
import random
import rasterio
from pathlib import Path
import splitfolders
import shutil
import math
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K
import time
from torch.nn.functional import interpolate
import onnx
from onnxconverter_common import auto_mixed_precision, auto_convert_mixed_precision, float16

4.33.3 0.23.0
2.0.1+cu118


## Check GPU Ressources

---



In [4]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri Sep 29 06:54:23 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8    12W /  70W |      3MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Unzip training data

---



In [5]:
%%capture
!unzip /content/gdrive/MyDrive/FLAIR2/flair_2_dataset/flair_2_aerial_test.zip
!unzip /content/gdrive/MyDrive/FLAIR2/flair_2_dataset/sentinel_mean_test.zip

## Split data for train/val

---



In [6]:
! mkdir "/content/temp"

# Chemin du dossier de destination
dst_folder = '/content/temp/'

# Chemin du dossier source
src_folder = '/content/flair_2_aerial_test'
for subdir, dirs, files in os.walk(src_folder):
    for file in files:
        src_file = os.path.join(subdir, file)
        dst_file = os.path.join(dst_folder, file)
        shutil.move(src_file, dst_folder)


# Chemin du dossier source
src_folder = 'content/data/sentinel'
for subdir, dirs, files in os.walk(src_folder):
    for file in files:
        src_file = os.path.join(subdir, file)
        dst_file = os.path.join(dst_folder, file)
        shutil.move(src_file, dst_folder)

# Chargement et conversion des modèles

---



In [ ]:
# CUDA DEVICE
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [7]:
# load LABEL DICT AND FEATURE EXTRACTOR
def array_to_dict(array):
    dictionary = {}
    for i, item in enumerate(array):
        dictionary[i] = item
    return dictionary

classes = ['None','building','pervious surface','impervious surface','bare soil','water','coniferous','deciduous','brushwood','vineyard','herbaceous vegetation','agricultural land','plowed land']
id2label = array_to_dict(classes)
label2id = {v: k for k, v in id2label.items()}

num_labels = len(id2label)

feature_extractor = SegformerFeatureExtractor(ignore_index=0, reduce_labels=False, do_resize=False, do_rescale=False, do_normalize=False)

/usr/local/lib/python3.10/dist-packages/transformers/models/segformer/feature_extraction_segformer.py:28: FutureWarning: The class SegformerFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use SegformerImageProcessor instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/segformer/image_processing_segformer.py:101: FutureWarning: The `reduce_labels` parameter is deprecated and will be removed in a future version. Please use `do_reduce_labels` instead.
  warnings.warn(


device(type='cuda')

In [8]:
pretrained_model_name_b5_sentinel_norm =  "/content/gdrive/MyDrive/FLAIR2/models/segformer_b5_rgb_norm_sentinel2-4e" #@param {type:"string"}
model_b5_sentinel_norm = SegformerForSemanticSegmentation.from_pretrained(
    pretrained_model_name_b5_sentinel_norm,
    id2label=id2label,
    label2id=label2id,
    reshape_last_stage=True,
    ignore_mismatched_sizes=True
)

model_b5_sentinel_norm  = model_b5_sentinel_norm.to(device)
dummy_input = torch.randn(1, 3, 512, 512, device="cuda")
torch.onnx.export(model_b5_sentinel_norm, dummy_input, "model_b5_sentinel_norm.onnx", verbose=True)

============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [9]:
pretrained_model_name_b5_igb_norm =  "/content/gdrive/MyDrive/FLAIR2/models/segformer_b5_igb_norm_aerial-8e+psdl" #@param {type:"string"}
model_b5_igb_norm = SegformerForSemanticSegmentation.from_pretrained(
    pretrained_model_name_b5_igb_norm,
    id2label=id2label,
    label2id=label2id,
    reshape_last_stage=True,
    ignore_mismatched_sizes=True
)

model_b5_igb_norm  = model_b5_igb_norm.to(device)
dummy_input = torch.randn(1, 3, 512, 512, device="cuda")
torch.onnx.export(model_b5_igb_norm, dummy_input, "model_b5_igb_norm.onnx", verbose=True)

============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [10]:
%%capture
#CONVERT TO ONNX FLOAT 16 // REDUCE THE SIZE OF THE MODEL BY 2

model = onnx.load("model_b5_sentinel_norm.onnx")
model_fp16 = float16.convert_float_to_float16(model)
onnx.save(model_fp16, "model_b5_sentinel_norm_fp16.onnx")

model = onnx.load("model_b5_igb_norm.onnx")
model_fp16 = float16.convert_float_to_float16(model)
onnx.save(model_fp16, "model_b5_igb_norm_fp16.onnx")

In [11]:
# Spécifiez les fournisseurs d'exécution que vous souhaitez utiliser
providers = ['TensorrtExecutionProvider', 'CUDAExecutionProvider', 'CPUExecutionProvider']

In [12]:
import onnxruntime as ort
ort.get_device()

'GPU'

In [13]:
ort_session_b5_sentinel_norm = ort.InferenceSession("model_b5_sentinel_norm_fp16.onnx", providers=providers)
ort_session_b5_igb_norm = ort.InferenceSession("model_b5_igb_norm_fp16.onnx", providers=providers)

EP Error /onnxruntime_src/onnxruntime/core/session/provider_bridge_ort.cc:1193 onnxruntime::Provider& onnxruntime::ProviderLibrary::Get() [ONNXRuntimeError] : 1 : FAIL : Failed to load library libonnxruntime_providers_tensorrt.so with error: libnvinfer.so.8: cannot open shared object file: No such file or directory
 when using ['TensorrtExecutionProvider', 'CUDAExecutionProvider', 'CPUExecutionProvider']
Falling back to ['CUDAExecutionProvider', 'CPUExecutionProvider'] and retrying.
EP Error /onnxruntime_src/onnxruntime/core/session/provider_bridge_ort.cc:1193 onnxruntime::Provider& onnxruntime::ProviderLibrary::Get() [ONNXRuntimeError] : 1 : FAIL : Failed to load library libonnxruntime_providers_tensorrt.so with error: libnvinfer.so.8: cannot open shared object file: No such file or directory
 when using ['TensorrtExecutionProvider', 'CUDAExecutionProvider', 'CPUExecutionProvider']
Falling back to ['CUDAExecutionProvider', 'CPUExecutionProvider'] and retrying.


In [14]:
!mkdir preds

def lister_images(dossier):
    chemins_images = []
    noms_images = []

    for dossier_racine, sous_dossiers, fichiers in os.walk(dossier):
        for fichier in fichiers:
            # Vérifie si le fichier se termine par ".tif"
            if fichier.endswith(".tif"):
                chemin_image = os.path.join(dossier_racine, fichier)
                nom_image = fichier

                chemins_images.append(chemin_image)
                noms_images.append(nom_image)

    return chemins_images, noms_images


# Spécifiez le chemin du dossier contenant les images
dossier_images = "/content/temp"

# Obtenez la liste des chemins et noms d'images
chemins, noms = lister_images(dossier_images)

# Inference

---



In [20]:
debut = time.time()
ix=0
for chemin, nom in zip(chemins, noms):
    if ix % 400 == 0:
        fin = time.time()
        temps_execution = int(fin - debut)
        print("nombre d'executions : ", ix, f" // temps d'execution : {temps_execution} secondes")

    ix= ix+1



    # get raster file
    with rasterio.open(chemin) as src_img:
        image = src_img.read([1,2,3]).swapaxes(0, 2).swapaxes(0, 1)
        igb = src_img.read([4,2,3]).swapaxes(0, 2).swapaxes(0, 1)
        igb = igb.astype(np.float32)

    # aerial igb normalization
    MEAN = np.array([ 0.40987858, 0.45704361, 0.42254708])
    STD = np.array([ 0.15510736, 0.1782405 , 0.17575739])
    igb_norm = aug.Compose([aug.Normalize(mean=MEAN, std=STD)])
    image_igb_norm = igb_norm(image=igb)['image']


    # sentinel normalization
    sen = np.load('/content/temp/'+ nom.replace('IMG','SEN').replace('tif','npy'))

    #get MEAN and STD for normalization
    MEAN = np.mean(sen.squeeze(), axis=(1, 2))
    STD= np.std(sen.squeeze(), axis=(1, 2))

    #Check for NaN values
    nan_present = any(math.isnan(x) for x in MEAN)
    if nan_present:
        MEAN = np.array([0.44050665, 0.45704361, 0.42254708])
        STD = np.array([0.20264351, 0.1782405 , 0.17575739])
    sentinel_norm = aug.Compose([aug.Normalize(mean=MEAN, std=STD)])
    image_sentinel_norm = sentinel_norm(image=image)['image']

    pixel_values_sentinel_norm = feature_extractor(image_sentinel_norm, return_tensors="np").pixel_values.astype(np.float16) #.to(device)
    pixel_values_igb_norm = feature_extractor(image_igb_norm, return_tensors="np").pixel_values.astype(np.float16) #.to(device)


    # image_np = pixel_values.cpu().numpy()
    output_b5_sentinel_norm = ort_session_b5_sentinel_norm.run(None,  {"input.1":pixel_values_sentinel_norm})[0]
    output_b5_igb_norm = ort_session_b5_igb_norm.run(None,  {"input.1":pixel_values_igb_norm})[0]


    ######################################################################################################
    # aggregate results predictions
    pred_segformer_b5_igb_norm = tf.image.resize(tf.transpose(output_b5_igb_norm, perm=[0,2,3,1]), size = [512,512], method="bilinear") # resize to 512*512
    pred_segformer_b5_sentinel_norm = tf.image.resize(tf.transpose(output_b5_sentinel_norm, perm=[0,2,3,1]), size = [512,512], method="bilinear") # resize to 512*512


    preds = np.mean(np.array([ pred_segformer_b5_igb_norm,pred_segformer_b5_sentinel_norm  ]), axis = 0)
    preds = [np.argmax(preds[index,:,:,:], axis = -1).transpose((0,1)) for index in range(preds.shape[0])]
    preds = np.squeeze(preds)
    preds = np.array(preds)-1
    preds = preds.astype('uint8')  # Pass prediction on CPU


    preds_sent = np.mean(np.array([pred_segformer_b5_sentinel_norm]), axis = 0)
    preds_sent = [np.argmax(preds_sent[index,:,:,:], axis = -1).transpose((0,1)) for index in range(preds_sent.shape[0])]
    preds_sent =np.squeeze(preds_sent)
    preds_sent = np.array(preds_sent)-1
    preds_sent = preds_sent.astype('uint8')  # Pass prediction on CPU

    preds_igb = np.mean(np.array([pred_segformer_b5_igb_norm]), axis = 0)
    preds_igb  = [np.argmax(preds_igb [index,:,:,:], axis = -1).transpose((0,1)) for index in range(preds_igb.shape[0])]
    preds_igb  =np.squeeze(preds_igb )
    preds_igb  = np.array(preds_igb )-1
    preds_igb  = preds_igb.astype('uint8')  # Pass prediction on CPU

    preds[preds == 5] = preds_igb[preds == 5]
    preds[preds == 8] = preds_sent[preds == 8]
    preds[preds == 9] = preds_igb[preds == 9]
    preds[preds == 10] = preds_igb[preds == 10]
    preds[preds == 11] = preds_igb[preds == 11]


    chemin_sortie = "/content/preds/" + nom.split('/')[-1].replace('IMG', 'PRED')
    Image.fromarray(preds).save(chemin_sortie, compression='tiff_lzw')

fin = time.time()
temps_execution = int((fin - debut)/60)

print(f" Temps total d'execution : {temps_execution} minutes")

nombre d'executions :  0  // temps d'execution : 0 secondes
nombre d'executions :  400  // temps d'execution : 67 secondes
nombre d'executions :  800  // temps d'execution : 134 secondes
nombre d'executions :  1200  // temps d'execution : 201 secondes
nombre d'executions :  1600  // temps d'execution : 270 secondes
nombre d'executions :  2000  // temps d'execution : 339 secondes
nombre d'executions :  2400  // temps d'execution : 408 secondes
nombre d'executions :  2800  // temps d'execution : 477 secondes
nombre d'executions :  3200  // temps d'execution : 546 secondes
nombre d'executions :  3600  // temps d'execution : 616 secondes
nombre d'executions :  4000  // temps d'execution : 685 secondes
nombre d'executions :  4400  // temps d'execution : 754 secondes
nombre d'executions :  4800  // temps d'execution : 823 secondes
nombre d'executions :  5200  // temps d'execution : 892 secondes
nombre d'executions :  5600  // temps d'execution : 961 secondes
nombre d'executions :  6000  // t

# Export results

---



In [21]:
!zip -r /content/gdrive/MyDrive/FLAIR2/outputs/flair2_T4_46_min_6355_iou.zip /content/preds

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
  adding: content/preds/PRED_083287.tif (deflated 2%)
  adding: content/preds/PRED_092974.tif (deflated 4%)
  adding: content/preds/PRED_085507.tif (deflated 13%)
  adding: content/preds/PRED_090822.tif (deflated 15%)
  adding: content/preds/PRED_085935.tif (deflated 10%)
  adding: content/preds/PRED_080977.tif (deflated 30%)
  adding: content/preds/PRED_078967.tif (deflated 1%)
  adding: content/preds/PRED_091052.tif (deflated 28%)
  adding: content/preds/PRED_078043.tif (deflated 8%)
  adding: content/preds/PRED_081110.tif (deflated 1%)
  adding: content/preds/PRED_089492.tif (deflated 2%)
  adding: content/preds/PRED_081527.tif (deflated 16%)
  adding: content/preds/PRED_087222.tif (deflated 71%)
  adding: content/preds/PRED_091865.tif (deflated 71%)
  adding: content/preds/PRED_087282.tif (deflated 71%)
  adding: content/preds/PRED_083059.tif (deflated 71%)
  adding: content/preds/PRED_085168.tif (deflate